# Math  1376: Programming for Data Science
---

In [ ]:
import numpy as np #We will use numpy in this lecture
import matplotlib.pyplot as plt
%matplotlib inline

## Module 04: Some useful applications of Modules 01-03
---

In this module, we will now pull together material across our first three modules to solve some practical problems. 

You may find it useful to review the notebooks in those modules beforehand or just simply open some of the notebooks to have their contents available to review as necessary. 

While there are a seemingly endless number of practical problems we can attempt to solve with what we have learned so far, we will focus on three ubiquitous problems in the computational sciences.

- Root-finding. (The content of Part (a) of this module.)
<br>

- Numerical integration. (The content of Part (b) of this module.)
<br>

- Optimization. (This content is pursued in the assignments of this module.)
<br>

### A note about calculus concepts and interactive visualizations (i.e., widgets)
---

These topics are commonly studied as applications of calculus concepts. 
However, while we may make passing reference to certain calculus concepts, you do *not* need to know calculus to follow the narratives. (This, of course, is not to say that you should not seek to master calculus at some point.)
Our focus is on the *big picture ideas* and we use interactive graphics (powered by a widgets module) to help us explore these ideas.

## Learning Objectives for Part (a)

- Understand what a root-finding problem means and how they arise in practice.


- Apply different types of root-finding algorithms to various functions written as `lambda` Python functions.


- Create code that implements root-finding algorithms of different types.


- Create annotations and interactive widgets to enhance visualizations of data.


### Some larger "in situ" learning objectives (i.e., learning that will occur by design of activities)

While we are going to explore how to implement some of these algorithms as activities below, our learning objectives go beyond simple correct implementation. We will also consider what it means to do the following:

- *compare* and *analyze* different algorithms developed for solving the same generic problem;
<br>

- use this comparison to *choose* the "right" algorithm for solving a *specific* problem;
<br>

- create a module that encodes various algorithms and a *wrapper* function that automatically chooses which algorithm to apply based on the inputs.
<br>

Some of this is done in the notebook while other parts are left for homework.

## Notebook contents <a id='Contents'>

* <a href='#Root-finding'>Part (a): Root-finding</a>
<br>

    * <a href='#Introduction'>Part (a)(i): An introduction to concepts</a>
    <br>
    
    * <a href='#Bracketing'>Part (a)(ii): Bracketing methods</a>
    <br>
        
        * <a href='#activity-bisection'>Activity: Implementing the bisection algorithm</a>
        <br>
        
        * <a href='#activity-bisection-variant'>Activity: Implementing a bisection variant</a>
        <br>
        
        * <a href='#activity-bracketing-compare'>Activity: Comparing bracketing methods</a>
        <br>
        
    * <a href='#Iterative'> Part (a)(iii): "Iterative" methods </a>
    <br>
    
        * <a href='#activity-secant'>Activity: Implementing the secant algorithm</a>
        <br>
        
        * <a href='activity-linear-methods-compare'>Activity: Comparing secant to false point</a>
        <br>
    
    * <a href='#activity-summary'>Activity: Summary</a>

## Part (a): Root-finding <a id='Root-finding'>
---
    
    
<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('_kROMq3pDzA', width=800, height=300)

## Part (a)(i): An introduction to concepts <a id='Introduction'>

### What does root-finding mean?

Simply put, given $f(x)$, a root of the function is some point $x=c$ such that $f(c)=0$. 

In other words, the ***roots*** of a function are the *inputs* that make the *output* equal to zero. 

### lambda functions for simple examples

We only mentioned these briefly in 03-Lecture-part-b.ipynb in the context of list comprehensions, so you may want to review that quickly.
They do prove to be useful things in the context of this notebook, so we dive into them a bit more here. 

What is a `lambda` function? You can read quite a bit about such functions [here](https://www.w3schools.com/python/python_lambda.asp) or [here](https://realpython.com/python-lambda/).

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- The main takeaway is that if you have a simple type of "anonymous" function that you just need for a short period of time in your code, then a lambda function is probably right for you. It is *not* necessary to use these, but they are useful. 
<br>

- A generic use syntax we care about would look something like this:

  > ```f = lambda arguments : expression ```

Run the code cells below to see an example of a `lambda` function in the context of plotting the roots of a polynomial.

Pay attention to how we use these types of functions in this lecture. They are completely anonymous functions that we pass as arguments into algorithms so that we can study how the algorithms perform on different types of functions without having to formally declare/create the functions elsewhere in the code.
In other words, they are simply a matter of convenience.

In [ ]:
f = lambda x : x**3 - x - 2  #defining f(x) = x^3-x-2

In [ ]:
x = np.linspace(-1,2,100)

plt.figure(figsize=(10,5))
plt.plot(x,f(x),'b',linewidth=2) #plot of function f
plt.axvline(0, linewidth=1, linestyle=':', c='k') #plot typical y-axis
plt.axhline(0, linewidth=1, linestyle=':', c='k') #plot typical x-axis

c = 1.521 #approximate root
plt.scatter(c,f(c),s=70,c='r',marker='s') #plot approximate root

###########################################
## Now for some fancier plotting options ##
###########################################

# Let's create a text string for the title that formats certain floating point numbers
title_str = 'Approx. root at c=' + '{:.3}'.format(c) + r' with f(c)$\approx$' + '{:.2e}'.format(f(c))
plt.title(title_str, fontsize=14)

# Now let's annotate the plot with some text and arrows

# First, we define the dict (dictionary) of properties of items used in the annotation
bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.5, linewidth=2) #This is the property dict for bounding box of text
arrowprops = dict(arrowstyle='->', color='k', alpha=.5, linewidth=2) #This is the property dict for the arrow

# Now annotate the plot
plt.annotate('Approx. root', fontsize=12, xy=(c,f(c)), xytext=(c-1.25,1), color='w',
             bbox = bbox, arrowprops=arrowprops);

***Of course, the above example just gives us more of the idea of the "what" but not the "why."***

### Why should we care about roots?

Is there anything particularly important about a function being zero? In general, the answer is no. However, it depends on the function and the goals of the problem/application for which the function models system behavior. 
The assignment discusses how root problems arise naturally in many optimization problems.
Below, we provide some simpler motivating examples.

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Some motivating examples:*** <span>

- Suppose there are two launched projectiles with two height functions denoted by $h_1(t)$ and $h_2(t)$, and we are very interested when, or even *if*, there is a time where $h_1(t)=h_2(t)$ (i.e., is there a time where the objects collide)? This is an important question that arises in missile defense systems, satellite monitoring, and autonomous vehicle guidance systems. Well, define $f(t)=h_1(t)-h_2(t)$. Guess what, now you are interested in knowing whether or not $f(t)=0$. 
<br>

- Suppose $p(t)$ describes a model you develop for the price of a potential investment (it could be in a savings/money market account, stock, or mutual fund to name just a few). You are probably interested in knowing when your model predicts the price to reach a certain target value, $p_{target}$, so that you can develop certain financial plans/goals around this time frame (e.g., having enough for a down payment on a car or house, take a vacation, or retire). Defining $f(t)=p(t)-p_{target}$ means you are interested in determining when $f(t)=0$.

### And these root problems are solved how?

There are lots of algorithms that attempt to approximate the roots of a function, e.g., see https://en.wikipedia.org/wiki/Root-finding_algorithms. 

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- Most algorithms are *iterative*. (Meaning we need to write *loops* to implement them.)
<br>

- Some algorithms are very easy to implement. Some are more difficult. Generally, the easier something is to implement, the more restrictive the conditions are under which we expect it to produce anything meaningful. 

## Part (a)(ii): Bracketing methods <a id='Bracketing'>
---

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the second recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('gvPYZdsLaBI', width=800, height=300)

## Bisection algorithm (https://en.wikipedia.org/wiki/Bisection_method)

---

The basic idea is best illustrated by an interactive demo and playing the video above.

***We use widgets to create the interactivity. Widgets are very useful for illustrating concepts and exploring ideas.***

In [ ]:
# Usually place these at the top of a notebook if you know you are going to use widgets
# This cell only needs to be executed once
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def compute_bisection(f, a, b, n=10, tol_interval=1e-5, tol_f=1e-8):
    '''This simple function applies up to n iterations of the bisection algorithm'''
    
    # Let us first check the conditions to apply the bisection alg. are satisfied
    if a >= b:
        raise ValueError('The bisec. alg. requires a<b')
    if f(a)*f(b) > 0:
        raise ValueError('The bisec. alg. requires f(a) and f(b) are different signs')
        
    # Now, perform the bisection algorithm
    current_iter = 0
    while current_iter < n:
        c = (b+a)/2 # bisect the interval [a,b], i.e., compute mid-point
        
        # Check if c is an (approx.) root
        if np.abs(f(c)) < tol_f:
            break
        
        # Check if dividing [a,b] in half makes the interval too small to continue
        if (b-a)/2 < tol_interval:
            break
            
        # Determine if a or b should be replaced with c
        if f(a)*f(c) < 0:
            b = c
        else:
            a = c
        
        current_iter += 1
    
    return (a, b, c)

In [ ]:
# This cell creates a plotting function that we use widgets to interact with in the following cell

def plot_bisection(f, a, b, n, x_min, x_max, grid_pts = 100, show_final_interval = False):
    # Compute approximate root with bisec. alg. 
    (a_n, b_n, c) = compute_bisection(f, a, b, n)
    
    ############################################
    # Everything below this line involves making plots of the results 
    ############################################
    fig = plt.figure(figsize=(10,5))
    x = np.linspace(x_min,x_max,grid_pts)
    
    # First, plot function and axes
    plt.plot(x,f(x),'b',linewidth=2) #plot of function f
    plt.axvline(0, linewidth=1, linestyle=':', c='k') #plot typical y-axis
    plt.axhline(0, linewidth=1, linestyle=':', c='k') #plot typical x-axis
    
    # Title information
    title_str = 'Approx. root is ' + '{:.6}'.format(c) + '. Use this to update interval [a,b].'
    plt.title(title_str, fontsize=14)
    
    # Create a dict (dictionary) of properties of items used in annotations
    bbox_pt = dict(boxstyle='round', fc='0.9')
    bbox_text = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2) 
    arrowprops = dict(arrowstyle='->', facecolor='black', alpha=0.5, linewidth=2) 
    y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
    y_range = y_max - y_min
    x_range = x_max - x_min
    
    # Now plot relevant points and annotate
    
    # First annotate parts of the plot involving the points a and b
    if show_final_interval:
        plt.scatter(a_n,f(a_n),s=70,c='k')
        
        plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(a_n,f(a_n)), xytext=(a_n+0.2*x_range,f(a_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b_n,f(b_n),s=70,c='k')
        
        plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(b_n,f(b_n)), xytext=(b_n+0.2*x_range,f(b_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
    else:
        plt.scatter(a,f(a),s=70,c='k')
        
        plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(a,f(a)), xytext=(a+0.1*x_range,f(a)-0.25*y_range),
                     bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b,f(b),s=70,c='k')
        
        plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(b,f(b)), xytext=(b+0.1*x_range,f(b)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)

    # Now annotate the part of the plot involving the approximate root c    
    plt.scatter(c,f(c),s=70,c='r')
    
    plt.annotate('Approx. root\nfrom ' + str(n) + '\nstep(s) of bisec.\nalgorithm', fontsize=12, xy=(c,f(c)), 
                     xytext=(c-0.2*x_range,f(c)+0.25*y_range), color='w',
                     bbox = bbox_text, arrowprops=arrowprops)
    
    plt.annotate(r'$(c,f(c))$', fontsize=12, xy=(c,f(c)), xytext=(c+0.1*x_range,f(c)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
  

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

interact(plot_bisection, 
         f = widgets.fixed(lambda x: x**3-x-2),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         grid_pts = widgets.fixed(100),
         show_final_interval = widgets.Checkbox(False))

In [ ]:
%reset -f out

interact(plot_bisection, 
         f = widgets.fixed(lambda x: np.sin(x**3-x-2)),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(3),
         grid_pts = widgets.fixed(1000),
         show_final_interval = widgets.Checkbox(False))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Playing with the bisection algorithm</span><a id='activity-bisection'>

Note: You may want to create additional code cells below as you work your way through this activity.

- Test the bisection plotting function on some `lambda` functions of your choosing both with and without widgets for interactivity.
<br><br>

- Create an `improved_plot_bisection` function that allows for all of the parameters in the `compute_bisection` function to be set by the user.
<br><br>

- Test your `improved_plot_bisection` function on some `lambda` functions of your choosing both with and without widgets where you show results for different choices of these new parameters.

<hr style="border:5px solid cyan"> </hr>

## A bisection variant (https://en.wikipedia.org/wiki/Regula_falsi)

The basic idea of *regula falsi* (i.e., the false position method) is quite simple. It follows 3 steps starting with an initial guess of an interval $[a,b]$ that may contain a root. 

1. Make a straight line between $(a,f(a))$ and $(b,f(b))$. 

2. Compute the $x$-intercept of this line and call that point $c$. 

3. Compute $f(c)$ and compare the sign to $f(a)$ (or $f(b)$) and update the interval to either $[a,c]$ or $[c,b]$ depending on the result of this comparison.

Thus, this is just a variant of the bisection algorithm. The fundamental difference is that instead of using the midpoint of the interval $[a,b]$ as the guess for the root $c$, we use *interpolation* to approximate the function with a simple function (a line) for which it is easy to determine the root. The exact root of this approximating function is then an approximate root of the exact function.

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Implementing a bisection variant</span><a id='activity-bisection-variant'>

Note: You may want to create additional code cells below as you work your way through this activity.

- Create a new function `plot_falsept` based on the `plot_bisection` function above to visualize the false point method using widgets. 
<br>

    - Visualize results with some `lambda` functions. 
<br>
<br>

- Create a function `my_falsept` that performs the false point method. Make sure it tests that the conditions are appropriate for running this algorithm. The output show include an annotated plot of results.
<br>

    - Test your `my_falsept` code on some `lambda` functions. 

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Comparing bracketing methods</span><a id='activity-bracketing-compare'>

Use a mixture of markdown and code cells below to compare the bisection algorithm and false point method. 

You may find the wiki articles useful to reference here as well as doing some searching on Google or your favorite search engine. 

Can you describe and implement problems where one method seemingly does better than another? Can you explain why that is happening? When would you consider using one over another? What questions about these methods and comparing them seem worth exploring to you? This is intentionally left as open as possible for you to pursue what you think are intriguing questions to pose, research, and answer.

<hr style="border:5px solid cyan"> </hr>

## Part (a)(iii) "Iterative" methods <a id='Iterative'>
---
    
<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the third recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('F-RLSHmC69o', width=800, height=300)

Iterative methods in root-finding problems generally refer to approaches that require an *initial guess of the root* instead of an initial interval/set containing the root. 
These methods generally rely on defining an *auxiliary* function (i.e., an approximation to the function) that is constructed at the current guess in order to form the next "best" guess at the root of the true function. 


I imagine the incredulity you must feel. After all, weren't the above methods iterative?
Well, yes they were in a very technical sense that they implement loops. 
In fact, the false-point method somewhat hints at the basic idea of these methods and looks similar to the *secant* method.
The difference is that bracketing methods also require a description of a set that contains the root you are looking for, which is significantly more information to specify at the start of a problem than just a single initial guess at the root.

One of the things you will need to consider, and which the instructions for the previous activity above hints at, is why some methods may be preferred to others in various scenarios. 

## The secant method (https://en.wikipedia.org/wiki/Secant_method)

This is perhaps best explained via an interactive demo and playing the video above. 

The wiki article actually includes Python code on this simple algorithm, which we copy below before the demo.

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Good questions to keep in mind:*** <span>

- Can you see where this code can immediately have some difficulty? 
<br>

- What would be a good initial check to ensure that the code does not "crash"? 
<br>

    - Should a similar check be done at the end of each iteration? 

In [ ]:
#Copied from the wiki article
def secant_method(f, x0, x1, iterations):
    """Return the root calculated using the secant method."""
    for i in range(iterations):
        x2 = x1 - f(x1) * (x1 - x0) / float(f(x1) - f(x0))
        x0, x1 = x1, x2
    return x2

In [ ]:
#Copied and edited from the wiki article for demo purposes
def secant_method_4_demo(f, x0, x1, iterations):
    """Return the root calculated using the secant method."""
    xs = [x0, x1]
    fs = [f(x0), f(x1)]
    for i in range(iterations):
        x2 = x1 - f(x1) * (x1 - x0) / float(f(x1) - f(x0))
        x0, x1 = x1, x2
        xs.append(x2)
        fs.append(f(x2))
    return xs, fs

In [ ]:
def plot_secant(f, x0, x1, n,  x_min, x_max, grid_pts = 100):
    # Perform the secant method
    xs, fs = secant_method_4_demo(f, x0, x1, n)
    
    ############################################
    # Everything below this line involves making plots of the results 
    ############################################
    fig = plt.figure(figsize=(15,7.5))
    x = np.linspace(x_min,x_max,grid_pts)
    
    # Plot function and axes
    plt.plot(x,f(x),'b',linewidth=2) #plot of function f
    plt.axvline(0, linewidth=1, linestyle=':', c='k') #plot typical y-axis
    plt.axhline(0, linewidth=1, linestyle=':', c='k') #plot typical x-axis
    
    # Title information
    title_str = 'The n=' + str(n) + r' iter. of the sec. mtd. gives $f(x_{n+1})\approx$' + '{:.2e}'.format(fs[-1])
    plt.title(title_str, fontsize=18)
    
    # Plot all the n secant lines
    for i in range(n):
        min_idx = np.argmin(xs[i:i+3])
        max_idx = np.argmax(xs[i:i+3])
        if (min_idx != 2) and (max_idx != 2):
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [fs[i:i+3][min_idx], fs[i:i+3][max_idx]],
                     'k', linewidth=1, alpha=0.5)
        elif min_idx == 2:
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [0, fs[i:i+3][max_idx]],
                     'k', linewidth=1, alpha=0.5)
        else:
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [fs[i:i+3][min_idx], 0],
                     'k', linewidth=1, alpha=0.5)
            
    # Make clear what the final interval is        
    plt.plot([xs[i], xs[i]], [0, fs[i]], 'r', linewidth=1, alpha=0.5)
    plt.plot([xs[i+1], xs[i+1]], [0, fs[i+1]], 'r', linewidth=1, alpha=0.5)    
    plt.scatter(np.array(xs),0*np.array(xs))
    
    # Annotate relevant points in the plot
    bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2)
    y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
    y_range = y_max - y_min
    for i in range(n+2):
        if fs[i] < 0:
            plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(xs[i],y_range*0.05), color='w', bbox = bbox)
        else:
            plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(xs[i],-y_range*0.05), color='w', bbox = bbox)

In [ ]:
interact(plot_secant,
         f = widgets.fixed(lambda x: x**3-x-2),
         x0 = widgets.FloatSlider(value=0.5, min=0, max=2.5, step=0.01),
         x1 = widgets.FloatSlider(value=2, min=0, max=2.5, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=10),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(2.5),
         grid_pts = widgets.fixed(1000))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Improving the secant algorithm</span><a id='activity-secant'>

Note: You may want to create additional code cells below as you work your way through this activity.

- Create a `my_secant` function that is *improved* from what is implemented in `plot_secant` in the sense that it checks for and avoids any potential errors that may occur during runtime. 
<br>

    - The output should include an annotated plot. 
    <br>
    
    - Test your code on some `lambda` functions.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Comparing secant to false point</span><a id='activity-linear-methods-compare'>

Use a mixture of code and markdown cells to compare the secant and false point methods across a variety of `lambda` functions. 

You may find the wiki articles useful to reference here as well as doing some searching on Google or your favorite search engine. 

Can you describe and implement problems where one method seemingly does better than another? Can you explain why that is happening? When would you consider using one over another? What questions about these methods and comparing them seem worth exploring to you? This is intentionally left as open as possible for you to pursue what you think are intriguing questions to pose, research, and answer.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Summary</span> <a id='activity-summary'/>

Summarize some of the key takeaways/points from this notebook in a list below and prepare a few code examples related to these takeaways/points in the code cells below. You need to have at least one examples for each of your summary points and you need at least three summary points.

In this notebook, we have seen the following:

- [Your summary point 1 goes here]


- [Your summary point 2 goes here]


- [Your summary point 3 goes here]

<hr style="border:5px solid cyan"> </hr>

### <a href='#Contents'>Click here to return to Notebook Contents</a>